In [2]:
import numpy as np
import pandas as pd

In [14]:
file_name = "task_log.csv"

df = pd.read_csv( file_name)

df.tail()

,date,start time,Task,Study,Duration (s)
3,07/04/2020,20:13:44,code,study,10
4,07/04/2020,20:15:18,code,study,3
5,07/04/2020,20:15:27,code,study,10
6,07/04/2020,20:15:27,code,study,0
7,07/04/2020,20:21:08,code,study,10


In [13]:
df.groupby(['Task','Study']).sum()

,,Duration (s)
Task,Study,
code,study,43
